# Aquarium Plan post-processing

In [49]:
# login to Benchling & Aquarium
from aqbt.tools import config_to_sessions
from aqbt.tools import parse_config
import toml

def config(config_path):
    with open(config_path, "r") as f:
        return parse_config(toml.load(f))


def sessions(config):
    return config_to_sessions(config)


sessions = sessions(config('config.toml'))

benchling = sessions['default']['benchling']
registry = sessions['default']['registry']
session = sessions['default']['aquarium']
sessions

{'default': {'aquarium': <AqSession(name=None, AqHTTP=<AqHTTP(user='vrana', url='http://52.27.43.242')>), parent=4491238136)>,
  'benchling': <benchlingapi.session.Session at 0x138075250>,
  'registry': <aqbt.aquarium.registry.LabDNARegistry at 0x138077760>}}

In [50]:
PLAN_ID = 39491
# http://52.27.43.242//plans?plan_id=39388
# session.Plan.find(39380)


In [51]:
plan = session.Plan.find(PLAN_ID)
plan.created_at

'2020-08-03T13:13:59.000-07:00'

## [optional] Pre-Processing

In [52]:
import functools
import networkx as nx

def add_pour_gel(planner):
    print("Adding pour gels")
    for op in planner.operations:
        if op.operation_type.name == 'Run Gel':
            fv = op.input('Gel')
            preds = planner.get_fv_predecessors(fv)
            if not preds:
                pour_gel = planner.create_operation_by_name("Pour Gel")
                planner.quick_wire(pour_gel, op)
                
def add_gblock_seq(planner):
    print("Adding gBlock sequences")
    for op in planner.operations:
        if op.operation_type.name == 'Order gBlock Fragment':
            fv = op.input('Bases')
            fv.value = op.outputs[0].sample.properties['Sequence']
            
def needs_seq_results(planner):
    print("Setting glycerol stock")
    for op in planner.operations:
        if op.operation_type.name == 'Make Glycerol Stock':
            fv = op.input('Needs Sequencing Results?')
            fv.value = 'Yes'
            
def urgent_primer(planner):
    print("Setting urgent primers")
    for op in planner.operations:
        if op.operation_type.name == 'Order Primer':
            fv = op.input('Urgent?')
            fv.value = 'yes'
            
def add_missing_synthesized_fragments(input_file, planner):
    print("Adding missing synthesized fragments")
    name_to_goal = {}

    for g in input_file['GOALS']:
        name_to_goal[g['SAMPLE']['query']['name']] = g

    fragment_stock = session.SampleType.find_by_name('Fragment Stock')

    missing_op = []
    for op in planner.operations:
        if op.operation_type.name == 'Assemble Plasmid':
            out = op.outputs[0].sample.name
            inputs = [fv.sample.name for fv in op.inputs]

            goal = name_to_goal[out]
            g = nx.DiGraph()
            g.add_edges_from(goal['EDGES'])
            expected_inputs = list(g.predecessors(out))
            missing = set(expected_inputs).difference(set(inputs))

            missing = missing.difference(['DH5alpha'])
            missing_op.append((op, missing))
    
    all_missing = functools.reduce(lambda x, y: x.union(y), [m[1] for m in missing_op])
    
    planner.browser.where({'name': list(all_missing)}, model_class='Sample')
    
    # we've already handled DH5alpha...
    for op, missing in missing_op:
        for m in missing:
            if 'synthesized' in m.lower():
                print('\t' + m)
                sample = planner.browser.where({'name': m}, model_class='Sample')[0]
                assert sample
                fv = planner.set_input_field_value_array(op, 'Fragment', sample, container=fragment_stock)
                _ops = planner.chain('Order gBlock Fragment', 'Rehydrate Fragment', fv)
                planner.set_field_value_and_propogate(_ops[0].outputs[0], sample)
            else:
                raise Exception("missing {}".format(m))
            

In [53]:
import json
from pydent.planner import Planner

with open('dirt.io.json', 'r') as f:
    input_file = json.load(f)
    
split_plans = []
with session.with_cache(timeout=120) as sess:
    print("PLAN {}".format(PLAN_ID))
    plan = sess.Plan.find(PLAN_ID)
    print(plan.name)
    print(plan.created_at)
    print(plan.updated_at)
    
    print('loading planner...')
    planner = Planner(plan)
    print('planner loaded...')
    
    add_missing_synthesized_fragments(input_file, planner)
    add_pour_gel(planner)
    add_gblock_seq(planner)
    needs_seq_results(planner)
    urgent_primer(planner)
    
    print("optimizing...")
    planner.optimize()
    
    print("splitting plans...")
    split_plans = planner.split()
    print('\tsplit into {} plans'.format(len(split_plans)))
    
    for i, p in enumerate(split_plans):
        p.name = 'Part {}/{}: {}'.format(i+1, len(split_plans), plan.name)
        print('Saving plan to server. Please be patient.')
        
        print('\tPlan: {}'.format(p.name))
        print('\tNumOps: {}'.format(len(p.operations)))
        print('\tprettifying...')
        p.prettify()
        p.save()
        print('\t' + p.url)

PLAN 39491
Plant TF__1596484868_(Terrarium v0.1.5)
2020-08-03T13:13:59.000-07:00
2020-08-03T13:14:10.000-07:00
loading planner...
planner loaded...
Adding missing synthesized fragments
	Synthesized_PlantTF_2020_Campaign__fdca00f2
	Synthesized_PlantTF_2020_Campaign__7ca02d59
	Synthesized_PlantTF_2020_Campaign__3478329a
	Synthesized_PlantTF_2020_Campaign__3478329a
	Synthesized_PlantTF_2020_Campaign__bfefbb49
	Synthesized_PlantTF_2020_Campaign__fdca00f2
	Synthesized_PlantTF_2020_Campaign__bfefbb49
	Synthesized_PlantTF_2020_Campaign__7ca02d59
	Synthesized_PlantTF_2020_Campaign__64fd711f
	Synthesized_PlantTF_2020_Campaign__bfefbb49
	Synthesized_PlantTF_2020_Campaign__64fd711f
	Synthesized_PlantTF_2020_Campaign__3478329a
	Synthesized_PlantTF_2020_Campaign__2356ca0e
	Synthesized_PlantTF_2020_Campaign__64fd711f
	Synthesized_PlantTF_2020_Campaign__2356ca0e
	Synthesized_PlantTF_2020_Campaign__fdca00f2
Adding pour gels
Adding gBlock sequences
Setting glycerol stock
Setting urgent primers
optimizi

In [77]:
def find_fragment_stocks(session, sample_id):
    return session.query({
        '__model__': "Item",
        '__query__': {
            'sample_id': sample_id,
            'object_type': {
                '__query__': {
                    'name': 'Fragment Stock'
                }
            },
            'location': {
                '__not__': 'deleted'
            }
        }
    })

with session.with_cache(timeout=60) as sess:
    plan = sess.Plan.find(39493)
    planner = Planner(plan)
    to_remove = []
    for op in planner.operations:
        if op.operation_type.name == 'Stitch by Overlap Extension':
            to_remove.append(op)
            successor = planner.get_fv_successors(op.outputs[0])[0]
            items = find_fragment_stocks(sess, successor.child_sample_id)
            planner.set_field_value(successor, item=items[0])
            planner.remove_wire(op.outputs[0], successor)
    planner.remove_operations(to_remove)
    planner.save()

In [74]:
planner.remove_operations?

Signature: planner.remove_operations(ops: Iterable[pydent.models.operation.Operation])
Docstring: <no docstring>
File:      ~/anaconda3/envs/aqbt/lib/python3.8/site-packages/pydent/planner/planner.py
Type:      method


In [ ]:
with session.with_cache(timeout=60) as sess:
    print('saving...')
    plan = sess.Plan.find(39489)
    planner = Planner(plan)
    status = set(op.status for op in planner.operations)
    plan.status = None
    plan.folder = 'SD2 Build'
    planner.save()
    print('done')

## Post

In [21]:
from pydent import Planner

with session.with_cache() as sess:
    
    planner = Planner(sess.Plan.find(39488))
    
    

In [78]:
import json
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

seqs = []

with session.with_cache() as sess:
    
    for plan_id in [39492, 39493]:
        planner = Planner(sess.Plan.find(plan_id))

        for op in planner.operations:
            if op.operation_type.name == 'Order gBlock Fragment':
                seq_str = op.outputs[0].sample.properties['Sequence']
                assert seq_str == op.inputs[0].value
                seq = SeqRecord(Seq(seq_str))
                seq.id = op.outputs[0].sample.name
                seqs.append(seq)

with open('syndna.fasta', 'w') as f:
    SeqIO.write(seqs, handle=f, format='fasta')

In [48]:
from dasi.utils.sequence.sequence_complexity import DNAStats

for seq in seqs:
    stats = DNAStats(str(seq.seq), 14, 20, 20)
    print(stats.cost())

0.15560640732265427
3.1258278145695337
2.066147859922181
1.7537437603993318
23.106976744186046
7.8264150943396205
23.824390243902442
22.755555555555556
24.851612903225814
22.728571428571428
23.7
23.53548387096774
8.022471910112364
23.03030303030303


In [28]:
import json
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

with open('design.out.json', 'r') as f:
    design_out = json.load(f)

In [32]:
seqs = []

for m in design_out['molecules']:
    if m['sequence']['bases'].lower() == seq.lower():
        print(m)

{'__name__': 'SHARED_SYNTHESIZED_FRAGMENT', '__index__': 179, '__type__': 'molecule', '__meta__': {}, 'sequence': {'bases': 'ATGGGTACCCGCCCATATGCTTGCCCTGTCGAGTCCTGCGATCGCCGCTTTTCTCGCCACGCCAATCTTACCCGCCATATCCGCATCCATACCGGTCAGAAGCCCTTCCAGTGTCGAATCTGCATGCGTAACTTCAGTCGTAATGCGAACCTTGTGCGCCACATCCGCACCCACACAGGATCCCAAAAGCCGTTCCAATGTCGGATCTGTATGCGGAACTTTAGTCGAAAGGCCGACCTGAGGCGTCACATTCGCACGCACACCGGCGAGAAGCCTTTTGCCTGTGACATTTGTGGGAGGAAGTTTGCCAGGAAGGGCGACCTCAAGAGGCATACCAAAATCCATACAGGTTAGGCGAATTTCTTATGATTTATGATTTTTATTATTAAATAAGTTATAAAAAAAATAAGTGTATACAAATTTTAAAGTGACTCTTAGGTTTTAAAACGAAAATTCTTATTCTTGAGTAACTCTTTCCTGTAGGTCAGGTTGCTTTCTCAGGTATAGCATGAGGTCGCTCTTATTGACCACACCTCTACCGGCATGCCGAGCAAATGCCTGCAAATCGCTCCCCATTTCTGATACCGTCGACCTCGAGTCA', 'length': 601, 'name': 'pMOD8_Backbone_weak_exp_strong_ad_zev', 'id': '<unknown id>', 'annotations': [{'start': 0, 'end': 351, 'strand': 1, 'color': '#0d7dc9', 'name': 'Zev4', 'type': 'Engineered_Regio'}, {'start': 351, 'end': 354, 'strand': 1, 'color': '#87ac24', 'name'

In [41]:
from dasi.utils.sequence.sequence_complexity import DNAStats

seq = 'AGTGTCCCTTAACCAGATTCGAAAAGCGGCCCTTAACCAGATTCGAAAAGCGGCAGTAATCTTTCGGTCTACGCAACTGACTAGCTAACTATCGTTTCGACTGGGCCAAGTAATCGGCCGACTTACGCAACTAGGTGTCAGTCTATGACACCGTTGTACTGGAGTAATCTATGCAGTTACGCAACTAAGATATAAGGCGCCTTTCCTGCCTCGCAAAGTAATCACGGTAGTTACGCAACTTGGTGAGATGGAAGGCCATCACCGGACGAGAGTAATCGCATCCTCTACGCAACTCTACCCGTACTGCTCCCTTGAGATAGCAAAAGTAATCAATACGCCTACGCAACTTGCCGTGTGCTGTGTTCAACACGATCGTCTAAAGTGAAAGTCGAGCTCGGTACAAAGTGAAAGTCGAGCTCGGTACGATCCT'

stats = DNAStats(seq, 14, 20, 20)

In [45]:
for d in design_out['designs'].values():
    for a in d['assemblies'][:1]:
        print(a['cost']['max synthesis complexity'])

36.29
1.48
2.07
7.83
8.02
36.29
80.99
3.13
1.75
7.83
8.02
2.07
1.75
7.83
8.02
1.75
1.48
81.16
7.83
81.16
3.13
3.13
7.83
7.83
2.07
37.35
1.48
7.83
8.02
7.83
